In [63]:
# import library
import pybamm
import numpy as np

In [93]:
def my_rk_ocp_positive(x, T):
    kb = 1.380649e-23
    m0 = 3.8724339986937832
    coeffs = [
        -0.38545242,
        0.07207664,
        0.08256196,
        -0.27144603,
        -0.41604248,
        1.38708789,
        1.11072656,
        -3.67485474,
        -1.02940457,
        4.24441946,
        0.24045833,
        -1.74123012,
    ]
    b = 0.88124821
    delx = 0.10572341
    x = 1 - x
    x = b * x + delx
    func = 0
    for i, a in enumerate(coeffs):
        # For i==0, the second term becomes zero (avoid potential issues with negative exponents)
        if i == 0:
            term = a * ((1 - 2 * x) ** (i + 1))
        else:
            term = a * (
                ((1 - 2 * x) ** (i + 1))
                - (2 * i * x * (1 - x) * ((1 - 2 * x) ** (i - 1)))
            )
        func += term
    return kb * T * np.log(x / (1 - x)) + func + m0


def my_rk_ocp_negative(x, T):
    kb = 1.380649e-23
    m0 = 0.11787222863387316
    coeffs = [
        0.10686728,
        0.04120991,
        -0.0697988,
        0.27019182,
        1.35141614,
        -1.64799623,
        -5.90470322,
        5.4219514,
        13.50384686,
        -7.70496124,
        -14.79406063,
        4.03252355,
        6.34233454,
    ]
    b = 0.67338648
    delx = 0.00612092
    x = b * x + delx
    func = 0
    for i, a in enumerate(coeffs):
        # For i==0, the second term becomes zero (avoid potential issues with negative exponents)
        if i == 0:
            term = a * ((1 - 2 * x) ** (i + 1))
        else:
            term = a * (
                ((1 - 2 * x) ** (i + 1))
                - (2 * i * x * (1 - x) * ((1 - 2 * x) ** (i - 1)))
            )
        func += term
    return kb * T * np.log(x / (1 - x)) + func + m0

In [94]:
# define class
class MySPMe(pybamm.BaseModel):
    def __init__(self):
        super().__init__()
        electrodes = ["negative", "positive"]  # two electrodes

        # define variable (i.e. concentration in two electrodes)
        c_i = [
            pybamm.Variable(
                f"{e.capitalize()} particle concentration", domain=f"{e} particle"
            )
            for e in electrodes
        ]

        # define parameters
        I = pybamm.Parameter("Current function [A]")
        D_i = [
            pybamm.Parameter(f"{e.capitalize()} electrode diffusivity")
            for e in electrodes
        ]
        R_i = [
            pybamm.Parameter(f"{e.capitalize()} particle radius") for e in electrodes
        ]
        c0_i = [
            pybamm.Parameter(f"Initial concentration in {e} electrode")
            for e in electrodes
        ]
        delta_i = [
            pybamm.Parameter(f"{e.capitalize()} electrode thickness")
            for e in electrodes
        ]
        A = pybamm.Parameter("Electrode width") * pybamm.Parameter("Electrode height")
        epsilon_i = [
            pybamm.Parameter(
                f"{e.capitalize()} electrode active material volume fraction"
            )
            for e in electrodes
        ]
        T = pybamm.Parameter("Ambient temperature")
        c_e = pybamm.Parameter("Initial concentration in electrolyte")
        k_i = [
            pybamm.Parameter(f"{e.capitalize()} electrode reaction rate")
            for e in electrodes
        ]
        c_i_max = [
            pybamm.Parameter(f"Maximum concentration in {e} electrode")
            for e in electrodes
        ]
        U_i = [
            pybamm.FunctionParameter(
                f"{e.capitalize()} electrode OCP [V]",
                {"stoichiometry": c_i[i] / c_i_max[i], "Temperature": T},
            )
            for (i, e) in enumerate(electrodes)
        ]
        # Constants
        F = pybamm.constants.F
        R = pybamm.constants.R

        # derive variables
        a_i = [3 * epsilon_i[i] / R_i[i] for i in [0, 1]]
        j_i = [I / a_i[0] / delta_i[0] / F / A, -I / a_i[0] / delta_i[0] / F / A]

        # governing equations
        N_i = [-D_i[i] * pybamm.grad(c_i[i]) for i in [0, 1]]
        dcdt_i = [-pybamm.div(N_i[i]) for i in [0, 1]]
        self.rhs = {c_i[i]: dcdt_i[i] for i in [0, 1]}

        # boundary conditions
        lbc = pybamm.Scalar(0)
        rbc = [-j_i[i] / D_i[i] for i in [0, 1]]
        self.boundary_conditions = {
            c_i[i]: {"left": (lbc, "Neumann"), "right": (rbc[i], "Neumann")}
            for i in [0, 1]
        }

        # initial condtions
        self.initial_conditions = {c_i[i]: c0_i[i] for i in [0, 1]}

        # define intermediate variables and OCP function parameters
        c_i_s = [pybamm.surf(c_i[i]) for i in [0, 1]]
        x_i_s = [c_i_s[i] / c_i_max[i] for i in [0, 1]]
        i_0_i = [
            k_i[i]
            * F
            * (
                pybamm.sqrt(c_e)
                * pybamm.sqrt(c_i_s[i])
                * pybamm.sqrt(c_i_max[i] - c_i_s[i])
            )
            for i in [0, 1]
        ]
        eta_i = [
            2 * R * T / F * pybamm.arcsinh(j_i[i] * F / (2 * i_0_i[i])) for i in [0, 1]
        ]

        # define output variables
        [U_n_plus_eta, U_p_plus_eta] = [pybamm.surf(U_i[i]) + eta_i[i] for i in [0, 1]]
        V = U_p_plus_eta - U_n_plus_eta
        self.variables = {
            "Time [s]": pybamm.t,
            "Voltage [V]": V,
            "Current [A]": I,
            "Negative particle concentration": c_i[0],
            "Positive particle concentration": c_i[1],
            "Negative particle surface concentration": c_i_s[0],
            "Positive particle surface concentration": c_i_s[1],
            "Positive electrode OCP [V]": U_i[1],
            "Negative electrode OCP [V]": U_i[0],
        }

In [95]:
# USe model
model = MySPMe()
# Imagine you've made RKOpenCircuitPotential
# model.submodels["Positive electrode OCP [V]"] = RK_Open_Circuit_Potential(model.param, "positive", "lithium-ion main", model.options, "primary")
# model.build_model()


electrodes = ["negative", "positive"]  # two electrodes
r_i = [
    pybamm.SpatialVariable("r", domain=[f"{e} partical"], coord_sys="spherical polar")
    for e in electrodes
]
R_i = [pybamm.Parameter(f"{e.capitalize()} particle radius") for e in electrodes]
geometry = {
    f"{e} particle": {r_i[i]: {"min": pybamm.Scalar(0), "max": R_i[i]}}
    for (i, e) in enumerate(electrodes)
}
domains = [f"{e} particle" for e in electrodes]
submesh_type = {d: pybamm.Uniform1DSubMesh for d in domains}
var_pts = {r: 20 for r in r_i}
param = pybamm.ParameterValues(
    {
        "Positive electrode diffusivity": 3.9 * 10**-14,
        "Negative electrode diffusivity": 3.9 * 10**-14,
        "Positive particle radius": 10 * 10 ** (-6),
        "Negative particle radius": 10 * 10 ** (-6),
        "Initial concentration in positive electrode": 9.9 * 10**4,
        "Initial concentration in negative electrode": 0.1 * 10**4,
        "Positive electrode thickness": 10 * 10 ** (-7),
        "Negative electrode thickness": 10 * 10 ** (-7),
        "Electrode width": 10**10 ** (-3),
        "Electrode height": 10**10 ** (-3),
        "Positive electrode active material volume fraction": 0.7,
        "Negative electrode active material volume fraction": 0.75,
        "Ambient temperature": 297,
        "Initial concentration in electrolyte": 2.5 * 10**4,
        "Positive electrode reaction rate": 2 * 10**-3,
        "Negative electrode reaction rate": 2 * 10**-3,
        "Maximum concentration in positive electrode": 10 * 10**4,
        "Maximum concentration in negative electrode": 10 * 10**4,
        "Current function [A]": -3,
        "Positive electrode OCP [V]": my_rk_ocp_positive,
        "Negative electrode OCP [V]": my_rk_ocp_negative,
    }
)

param.process_geometry(geometry)

mesh = pybamm.Mesh(geometry, submesh_type, var_pts)
spatial_methods = {d: pybamm.FiniteVolume() for d in domains}
disc = pybamm.Discretisation(mesh, spatial_methods)

param.process_model(model)
disc.process_model(model)
solver = pybamm.ScipySolver()

In [96]:
t_eval = np.linspace(0, 1800, 1000)
solution = solver.solve(model, t_eval)

In [97]:
solution.plot(
    [
        "Voltage [V]",
        "Current [A]",
        "Negative particle concentration",
        "Positive particle concentration",
        "Negative particle surface concentration",
        "Positive particle surface concentration",
        "Positive electrode OCP [V]",
        "Negative electrode OCP [V]",
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=1800.0, step=18.0), Output()), _dom_classes=…